In [1]:
from device import Device
from rgf import GreensFunction
import scipy as sp
from hamiltonian import Hamiltonian
from helper import Helper_functions
import scipy.sparse as spa
import numpy as np
import scipy.sparse as sp
from lead_self_energy import LeadSelfEnergy
from scipy.sparse import bmat, identity, random
from scipy.sparse.linalg import eigsh, eigs
import time

In [9]:
device = Device()
ham = Hamiltonian(device)
lse = LeadSelfEnergy(device, ham)
gf = GreensFunction(device, ham)

G_R, gamma1, gamma2, sigmaL, sigmaR = gf.sparse_rgf_G_R(0.1, 0.1)



(140, 6)
(140, 6)
Hamiltonian construction: 0.7412052154541016 
               Self Energy construction: 0.5562310218811035 
                   Forward iteration: 0.629638671875 
                       Backward iteration: 0.06818222999572754


In [ ]:
sigma = lse.iterative_self_energy()

In [ ]:
from poisson import PoissonSolver

poissonSolver = PoissonSolver(device)

In [ ]:
poissonSolver.solve_poisson_equation()



In [ ]:
device = Device()
ham = Hamiltonian(device)
rgf = GreensFunction(device, ham)

G_R, gamma1, gamma2, sigmaL, sigmaR= rgf.sparse_rgf_G_R(E=0.1, ky = 0.1)


In [ ]:
print(G_R[0].shape)

In [ ]:
time_start = time.time()
dagger = lambda A: np.conjugate(A.T)
side = "left"
Hpp_matrices = [None] * lead.P
HpP_matrices = [None] * lead.P
hPP,hPP1 = lead.get_layer_hamiltonian(lead.P, side)

HPP = Helper_functions.sparse_inverse(spa.csc_matrix(lead.E * np.eye(hPP.shape[0]) - hPP))
Hpp_matrices[-1], HpP_matrices[-1] = HPP, HPP
for i in range(lead.P - 1, 0, -1):
    
    hpp, hpp1 = lead.get_layer_hamiltonian(i, side)
    Hpp = Helper_functions.sparse_inverse(spa.csc_matrix(lead.E * np.eye(hPP.shape[0]) - \
        hpp - hpp1 @ Hpp_matrices[i] @ dagger(hpp1)))
    Hpp_matrices[i - 1] = Hpp
    HpP = Hpp_matrices[i - 1] @ hpp1 @ HpP_matrices[i]
    HpP_matrices[i - 1] = HpP
    
C22 = Hpp_matrices[1]
C2P = HpP_matrices[1]
C_matrices = [None] * 4
C_matrices[1] = C22
for p in range(3, lead.P + 1):
    hpp, hpp1 = lead.get_layer_hamiltonian(i, side)
    C_matrices[p - 1] = Hpp_matrices[p - 1] + Hpp_matrices[p -1] @ (hpp1 @ C_matrices[p -2] @ dagger(hpp1)) @ Hpp_matrices[p - 1] 

h11, h12 = lead.get_layer_hamiltonian(1, side)

XIs = h11 + h12 @ C_matrices[1] @ dagger(h12)
XI = XIs + dagger(hPP1) @ C_matrices[-1] @ hPP1
PI = h12 @ C2P @ hPP1





In [ ]:
XIs = spa.csc_matrix(XIs)
XI = spa.csc_matrix(XI)
PI = spa.csc_matrix(PI)
I = np.eye(XI.shape[0], dtype=XI)
Z = I * 0
D = lead.E * I.copy() - XI
T = -PI

A = bmat([
    [Z, I],
    [-T.conj().T, -D]
], format='csc')

B = bmat([
    [I, Z],
    [Z, T]
], format='csc')

eigenvalues, eigenvectors = eigs(A, M=B, sigma=1.0, which='LM')




def construct_U_plus_and_Lambda_plus(eigenvalues, eigenvectors, n_dim, epsilon=0.1):
    abs_vals = np.abs(eigenvalues)
    

    is_propagating = np.isclose(abs_vals, 1.0)
    is_evanescent = (abs_vals < 1.0) & (abs_vals > epsilon)
    
    selected_indices = np.where(is_propagating | is_evanescent)[0]
    
    if len(selected_indices) == 0:
        return np.array([], dtype=complex), np.array([],dtype=complex)
        
    filtered_eigenvalues = eigenvalues[selected_indices]
    filtered_eigenvectors = eigenvectors[:, selected_indices]

    Lambda_plus = np.diag(filtered_eigenvalues)
    U_plus = filtered_eigenvectors[:n_dim, :]

    return U_plus, Lambda_plus

U_plus, Lambda = construct_U_plus_and_Lambda_plus(eigenvalues, eigenvectors, T.shape[0], epsilon=0.1)

U_pseudo = np.linalg.pinv(U_plus)
F = U_plus @ Lambda @ U_pseudo

Y = np.linalg.solve(lead.E * I - XIs.toarray() - PI.toarray() @ F, dagger(h12.toarray()))
self_energy = h12 @ Y
time_end = time.time()
print(f"time is: {time_end - time_start}")

In [ ]:
print(self_energy)

In [ ]:
H00,H10 = device.hamiltonian.get_H00_H01(ky=0.1, sparse=True)
print(H00)

In [ ]:
H00,H10 = device.hamiltonian.get_H00_H01(ky=0.1, sparse=False)
print(H00)

In [ ]:
def surface_gf(Energy, H00, H10, tol=1e-6): 
    """ 
    This iteratively calculates the surface green's function for the lead based. 
    Although it is tested for 1D, it should be good for 2D surfaces. 
    """

    Energy = Energy
    dagger = lambda A: np.conjugate(A.T)
    
    I = np.eye(H00.shape[0], dtype=complex)
    H01 = dagger(H10)

    epsilon_s = H00.copy()
    epsilon = H00.copy()
    alpha = H01.copy()
    beta = dagger(H10).copy()
    err = 1.0
    first_time = True

    while err > tol:
        if first_time:
            inv_E = Helper_functions.sparse_inverse(spa.csr_matrix(Energy * I) - spa.csr_matrix(epsilon))
            first_time = False
        else:

            inv_E = np.linalg.solve(Energy * I - epsilon, I)
    
        epsilon_s_new = epsilon_s + alpha @ inv_E @ beta
        epsilon_new = epsilon + beta @ inv_E @ alpha + alpha @ inv_E @ beta
        alpha_new = alpha @ inv_E @ alpha
        beta_new = beta @ inv_E @ beta

        err = np.linalg.norm(alpha_new, ord='fro')

        epsilon_s, epsilon, alpha, beta = epsilon_s_new, epsilon_new, alpha_new, beta_new

    return  np.linalg.solve(Energy * I - epsilon_s, I)
H00,H10 = device.hamiltonian.get_H00_H01(ky=0.1, sparse=True)


surface_gf(0, H00, H10, tol=1e-3)

In [ ]:
def surface_gf(Energy, H00 : np.ndarray, H10: np.ndarray, tol=1e-6): 
    """ 
    This iteratively calculates the surface green's function for the lead based. 
    Although it is tested for 1D, it should be good for 2D surfaces. 
    """
    
    Energy = Energy
    dagger = lambda A: np.conjugate(A.T)
    
    I = np.eye(H00.shape[0], dtype=complex)
    H01 = dagger(H10)

    epsilon_s = H00.copy()
    epsilon = H00.copy()
    alpha = H01.copy()
    beta = dagger(H10).copy()
    err = 1.0

    while err > tol:
        inv_E = np.linalg.solve(Energy * I - epsilon, I)

        epsilon_s_new = epsilon_s + alpha @ inv_E @ beta
        epsilon_new = epsilon + beta @ inv_E @ alpha + alpha @ inv_E @ beta
        alpha_new = alpha @ inv_E @ alpha
        beta_new = beta @ inv_E @ beta

        err = np.linalg.norm(alpha_new, ord='fro')

        epsilon_s, epsilon, alpha, beta = epsilon_s_new, epsilon_new, alpha_new, beta_new

    return  np.linalg.solve(Energy * I - epsilon_s, I)

H00,H10 = device.hamiltonian.get_H00_H01(ky=0.1, sparse=False)

surface_gf(0, H00, H10)

In [ ]:
from helper import Helper_functions

H00,H10 = device.hamiltonian.get_H00_H01(ky=0.1, sparse=True)
print(H00)
Helper_functions.sparse_inverse(-H00)
H00,H10 = device.hamiltonian.get_H00_H01(ky=0.1, sparse=False)
print(H00)
Helper_functions.sparse_inverse(-H00)

In [ ]:
H = device.hamiltonian.create_sparse_hamlitonian(0.1)